## Prepare soil data for use in LIM 
<br>

<b>Primary Author:</b>  Meg D. Fowler, CIRES/NOAA PSD <br>
<b>Date:</b>    6 Feb 2020 <br>
<b>Name:</b>    Prepare soil data

<br>
<b>Short Description</b> <br>
- Read in soil temperature and moisture observations from files containing NOAA HMT and US CRN data. <br>
- Center the data by removing the mean <br>
- Remove the average annual cycle from the data linearly <br>
- Get z-scores of soil anomalies <br>
- Linear regression between soil temperature and moisture to isolate only *residual* soil temperature <br>
<br><br>
Station order: [CZC, HBG, LSN, PTV, ROD, WLS, BDG, MRC, RED, STB, YSM]<br>
<br>Note: This script is used for producing data in <i>Fowler and Penland</i> (2020; in prep).<br><br>


### Import libraries

In [9]:
import numpy as np 
from   numpy import linalg as LA
import pandas as pd
from   datetime import datetime
from   dateutil import tz
import pickle 


In [2]:
# Define some basics 
stationIDs = ['CZC','HBG','LSN','PTV','ROD','WLS','BDG','MRC','RED','STB','YSM']


## 1. Read in soil data

- Data is saved locally on NOAA machines, so paths will not work outside of this access. Note also that NOAA HMT data is not publicly available, but US-CRN data is. <br>
- HMT data is saved in complex ways such that a single station may have multiple files for a single year, often reflecting added instrumentation. Thus a somewaht complex function is written to handle each case specifically. <br>
- Times are converted from UTC to local California time for both soil networks.

### 1.1 Define functions to read data

In [2]:
#Define function to move from UTC to local California time 

def getLocTime(calenDay,year,time):
    localDate  = np.empty(len(time), dtype='datetime64[s]')     #Empty array to save local time into 
    
    # Settings for time conversion 
    from_zone = tz.gettz('UTC')                     #Time zone data is currently saved in
    to_zone   = tz.gettz('America/Los_Angeles')     #Time zone you *want* data to be in instead 
    
    #-- Convert UTC to Pacific Time --#
    for iT in np.arange(0,len(time)):
        strDay  = str(int(np.asarray(calenDay[iT]))).zfill(3)
        strYr   = str(int(np.asarray(year[iT])))
        strHr   = str(int(np.asarray(time[iT]))).zfill(4)[:2]
        
        utc     = datetime.strptime(strYr+' '+strDay+' '+strHr, '%Y %j %H')
        # Tell the datetime object that it's in UTC time zone  
        utc     = utc.replace(tzinfo=from_zone)
        # Convert time zone
        pacTime = utc.astimezone(to_zone)
        
        # Save local date for given time 
        localDate[iT] = pacTime.replace(tzinfo=None)
    # ------- 
    
    return localDate


In [3]:
#Define function to read in soil MOISTURE from the US-CRN network files

def readData_CRN(fileName):
    # Read in file data
    smData         = pd.read_csv(fileName, sep=",", header=0,skiprows=1)
    smData.columns = ["network","stationID","Year","Month","Day","Doy","sm_5cm","sm_10cm",
                      "sm_20cm","sm_50cm","sm_100cm","Lat","Lon","flag_depth1","flag_depth2",
                      "flag_depth3","flag_depth4","flag_depth5"]
    
    # Get date as a single value
    dt = np.empty(len(smData.Year), dtype='datetime64[s]')
    for iT in range(len(smData.Year)):
        dt[iT] = datetime(year=int(smData.Year[iT]), month=int(smData.Month[iT]), day=int(smData.Day[iT]))

    # Add local time as column to pandas DF and drop year/month/day
    smData.insert(0,"Date",dt,True)
    smData = smData.drop('Year', axis=1)
    smData = smData.drop('Month', axis=1)
    smData = smData.drop('Day', axis=1)
    
    return smData


In [4]:
# Define function to return soil TEMPERATURE from CRN network in addition to soil moisture, which is returned above

def readAllCRN(stationSpec): 
    mainPath = '/home/mfowler/Python/Data/CRN_CAdata/OriginalFiles/CRND0103-'

    yrRange  = np.arange(2011,2020)

    appended_data = []
    for iYr in range(len(yrRange)):
        fileName = mainPath+str(yrRange[iYr])+'-CA_'+stationSpec+'.txt'
        yrData   = pd.read_csv(fileName,sep='\s+',header=None)
        yrData.columns = ["WBANNO","Date","CRX_VN","Lon","Lat","T_Daily_Max","T_Daily_Min","T_Daily_Mean",
                         "T_Daily_Avg","P_Daily_Calc","SolaRad_Daily","Sfc_T_Daily_Type","Sfc_T_Daily_Max",
                         "Sfc_T_Daily_Min","Sfc_T_Daily_Avg","RH_Daily_Max","RH_Daily_Min","RH_Daily_Avg",
                         "SM_5_Daily","SM_10_Daily","SM_20_Daily","SM_50_Daily","SM_100_Daily",
                         "ST_5_Daily", "ST_10_Daily","ST_20_Daily","ST_50_Daily","ST_100_Daily"]
        if iYr==0:
            appended_data = yrData
        else:
            appended_data = pd.concat([appended_data, yrData],sort=False, ignore_index=True)
            
    #Put date in datetime form
    appended_data.Date = pd.to_datetime(appended_data.Date,format = "%Y%m%d")

    return appended_data
    

In [5]:
# Define function to return data for selected stations from the NOAA HMT files 
#    Note: These are a bit messier, but this seems like the cleanest way to organize it all. 
#    Station options: czc, hbg, lsn, mdt, ptv, pvc, pvw, rod, rve, rvn, rvw, str, wls
#   "splitYear" indicates the year in which two records are present. Set to 0 to use only default section or if no split.

def readData_Russian(station,stationYears,splitYear):

    # Set paths 
    mainPath = '/psd2data/Soil_Moisture_qc1/'
    hrPath   = '/hourly/'

    #Initialize arrays to save all the data at this station
    appended_data = []

    #Loop over years in selection 
    for iYr in np.arange(0,len(stationYears)):
        
        # --- Define filenames (many stations have special cases) and column names ---
        if station=='czc':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_5","soilT_10","soilT_15","soilT_20","soilT_50","soilT_100",
                    "soilM_5","soilM_10","soilM_15","soilM_20","soilM_50","soilM_100"] 
            if iYr==0:
                yrString = '2012B'
                fileName = mainPath+station+hrPath+station+yrString+'.dat'
            else:
                fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
        
        elif station=='hbg':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15",
                    "soilM_10","soilM_15"]
            if stationYears[iYr]==splitYear:                    #2013 is split in two
                yrString = '2013A'
                fileName = mainPath+station+hrPath+station+yrString+'.dat'
                
                colNames2 = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15","soilT_50",
                    "soilM_10","soilM_15","soilM_50"]
                yrString2 = '2013B'   
                fileName2 = mainPath+station+hrPath+station+yrString2+'.dat'
            else:
                fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
        
        elif station=='lsn':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15",
                    "soilM_10","soilM_15"]
            if stationYears[iYr]==2017:                    #2017 is split in two
                yrString = '2017A'   
                fileName = mainPath+station+hrPath+station+yrString+'.dat'
                
                colNames2 = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15","soilT_50",
                    "soilM_10","soilM_15","soilM_50"]
                
                yrString2 = '2017B'
                fileName2 = mainPath+station+hrPath+station+yrString2+'.dat'
            else:
                fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
        
        elif station=='mdt':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_5","soilT_10","soilT_15","soilT_20","soilT_30",
                    "soilM_5","soilM_10","soilM_15","soilM_20","soilM_30"]
            fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
            
        elif station=='ptv':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15",
                    "soilM_10","soilM_15",]
            if stationYears[iYr]==2017:                    #2017 is split in two
                yrString = '2017A'   
                fileName = mainPath+station+hrPath+station+yrString+'.dat'
                
                colNames2 = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15","soilT_50",
                    "soilM_10","soilM_15","soilM_50"]
                yrString2  = '2017B'
                fileName2 = mainPath+station+hrPath+station+yrString2+'.dat'
            else:
                fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
        
        elif station=='pvc':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_5","soilT_10","soilT_15","soilT_20","soilT_30",
                    "soilM_5","soilM_10","soilM_15","soilM_20","soilM_30"]
            fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
        
        elif station=='pvw':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_5","soilT_10","soilT_15","soilT_20","soilT_50","soilT_100",
                    "soilM_5","soilM_10","soilM_15","soilM_20","soilM_50","soilM_100"]
            fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
            
        elif station=='rod':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15",
                    "soilM_10","soilM_15"] 
            if stationYears[iYr]==2017:                    #2017 is split in two
                yrString = '2017A'   
                fileName = mainPath+station+hrPath+station+yrString+'.dat'
                
                colNames2 = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15","soilT_50",
                    "soilM_10","soilM_15","soilM_50"] 
                yrString2 = '2017B'
                fileName2 = mainPath+station+hrPath+station+yrString2+'.dat'
            else:
                fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
                
        elif station=='rve':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_5","soilT_10","soilT_15","soilT_20","soilT_50","soilT_100",
                    "soilM_5","soilM_10","soilM_15","soilM_20","soilM_50","soilM_100"]
            fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
            
        elif station=='rvn':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_5","soilT_10","soilT_15","soilT_20","soilT_50","soilT_100",
                    "soilM_5","soilM_10","soilM_15","soilM_20","soilM_50","soilM_100"] 
            fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
        elif station=='rvw':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_5","soilT_10","soilT_15","soilT_20","soilT_50","soilT_100",
                    "soilM_5","soilM_10","soilM_15","soilM_20","soilM_50","soilM_100"] 
            fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
        elif station=='str':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15","soilT_20","soilT_50","soilT_100",
                    "soilM_10","soilM_15","soilM_20","soilM_50","soilM_100"] 
            if stationYears[iYr]==2017: 
                yrString = '2017A'   
                fileName = mainPath+station+hrPath+station+yrString+'.dat'
                
                colNames2 = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_5","soilT_10","soilT_15","soilT_20","soilT_50","soilT_100",
                    "soilM_5","soilM_10","soilM_15","soilM_20","soilM_50","soilM_100"] 
                yrString2 = '2017B'
                fileName2 = mainPath+station+hrPath+station+yrString2+'.dat'
            else: 
                fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'
        elif station=='wls':
            colNames = ["year","calenDay","time","sfcT","RH","P",
                    "soilT_10","soilT_15",
                    "soilM_10","soilM_15",]
            fileName = mainPath+station+hrPath+station+str(stationYears[iYr])+'.dat'     
        # -------------------------        
        
        #If block to handle potential case where data record is split up 
        if (splitYear==0 or stationYears[iYr]<splitYear): #Don't worry about split

            #Read in file as a pandas dataframe 
            data         = pd.read_csv(fileName, sep=",", header=None, usecols=(np.arange(1,len(colNames)+1)))
            data.columns = colNames
        
             # Get local time instead of UTC
            localDate = getLocTime(data.calenDay,data.year,data.time)

            # Add local time as column to pandas DF
            data.insert(0,"LocalTime",localDate,True)


            # Append this year's data onto the end of the last year
            if iYr==0:
                appended_data = data
            elif iYr>0:
                appended_data = pd.concat([appended_data, data],sort=False, ignore_index=True)

        elif stationYears[iYr]==splitYear:         #Special case for year of data split
            # --- First half of year --- #
            
            #Read in file as a pandas dataframe 
            data         = pd.read_csv(fileName, sep=",", header=None, usecols=(np.arange(1,len(colNames)+1)))
            data.columns = colNames
            
            # Get local time instead of UTC
            localDate = getLocTime(data.calenDay,data.year,data.time)

            # Add local time as column to pandas DF
            data.insert(0,"LocalTime",localDate,True)

            #appended_data.append(data)
            appended_data = pd.concat([appended_data, data],sort=False, ignore_index=True)

            # --- Second half of year --- #
            
            #Read in file as a pandas dataframe 
            data         = pd.read_csv(fileName2, sep=",", header=None, usecols=(np.arange(1,len(colNames2)+1)))
            data.columns = colNames2
            
            # Get local time instead of UTC
            localDate = getLocTime(data.calenDay,data.year,data.time)

            # Add local time as column to pandas DF
            data.insert(0,"LocalTime",localDate,True)

            #appended_data.append(data)
            appended_data = pd.concat([appended_data, data],sort=False, ignore_index=True)

        elif (stationYears[iYr]>splitYear): 
            data         = pd.read_csv(fileName, sep=",", header=None, usecols=(np.arange(1,len(colNames2)+1)))
            data.columns = colNames2
            
            # Get local time instead of UTC
            localDate = getLocTime(data.calenDay,data.year,data.time)

            # Add local time as column to pandas DF
            data.insert(0,"LocalTime",localDate,True)

            #appended_data.append(data)
            appended_data = pd.concat([appended_data, data],sort=False, ignore_index=True)

            
    #Create one giant pandas dataframe for whole period 

    #Remove any missing values (typically stored as -1000 or -999, but if SM<0, you know it's wrong [Bob's method])
    appended_data.loc[(appended_data.soilM_15<0), :] = np.nan
    appended_data.loc[(appended_data.soilM_15>2), :] = np.nan  #Really weird outlier data point
    #Add second flag dependent on temperature - some weird stuff's when I plot it... 
    appended_data.loc[(appended_data.sfcT<-500), :] = np.nan
    #Add another flag for precipitation -_- 
    appended_data.loc[(appended_data.P<0), :]    = np.nan
    appended_data.loc[(appended_data.P>4000), :] = np.nan  #Remove bad data point in HBG
    
    return appended_data


### 1.2 Read in the actual data

Read in NOAA HMT data, which is saved in individual (1-2) yearly files. 

In [6]:
#Station: CZC
appended_data_czc = readData_Russian('czc',np.arange(2012,2018),0)
dailyAvg_czc      = appended_data_czc.resample('D', on = 'LocalTime').mean()   #Get daily average 
print('Done with CZC')

#Station: HBG
appended_data_hbg = readData_Russian('hbg',np.arange(2006,2018),2013)
dailyAvg_hbg      = appended_data_hbg.resample('D', on = 'LocalTime').mean()   #Get daily average 
print('Done with HBG')

#Station: LSN
appended_data_lsn = readData_Russian('lsn',np.arange(2010,2018),2017)
dailyAvg_lsn      = appended_data_lsn.resample('D', on = 'LocalTime').mean()   #Get daily average 
print('Done with LSN')

#Station: PTV
appended_data_ptv = readData_Russian('ptv',np.arange(2011,2018),2017)
dailyAvg_ptv      = appended_data_ptv.resample('D', on = 'LocalTime').mean()   #Get daily average 
print('Done with PTV')

#Station: ROD
appended_data_rod = readData_Russian('rod',np.arange(2006,2018),2017)
dailyAvg_rod      = appended_data_rod.resample('D', on = 'LocalTime').mean()   #Get daily average 
print('Done with ROD')

#Station: WLS
appended_data_wls = readData_Russian('wls',np.arange(2010,2018),0)
dailyAvg_wls      = appended_data_wls.resample('D', on = 'LocalTime').mean()   #Get daily average 
print('Done with WLS')



Done with CZC
Done with HBG
Done with LSN
Done with PTV
Done with ROD
Done with WLS


Read in US CRN data. Soil moisture is read in first, as it retrieved from the National Soil Moisture Network. To look at soil temperature as well, we read in the raw CRN files second.

In [ ]:
#Define file paths
mainPath      = '/home/mfowler/Python/Data/CRN_CAdata/'

BodegaFile    = mainPath+'CRNH0203-2018-CA_Bodega_6_WSW.csv'
MercedFile    = mainPath+'CRNH0203-2018-CA_Merced_23_WSW.csv'
ReddingFile   = mainPath+'CRNH0203-2018-CA_Redding_12_WNW.csv'
SBfile        = mainPath+'CRNH0203-2018-CA_Santa_Barbara_11_W.csv'
YosemiteFile  = mainPath+'CRNH0203-2018-CA_Yosemite_Village_12_W.csv'

#Read in data for each station 
sm_Bodega    = readData_CRN(BodegaFile)
sm_Merced    = readData_CRN(MercedFile)
sm_Redding   = readData_CRN(ReddingFile)
sm_SantaBarb = readData_CRN(SBfile)
sm_Yosemite  = readData_CRN(YosemiteFile)


In [ ]:
# Read in the full CRN files for each station (to grab temperature)
BDG_allCRN = readAllCRN('Bodega_6_WSW')
MRC_allCRN = readAllCRN('Merced_23_WSW')
RED_allCRN = readAllCRN('Redding_12_WNW')
STB_allCRN = readAllCRN('Santa_Barbara_11_W')
YSM_allCRN = readAllCRN('Yosemite_Village_12_W')


### 1.3 Restrict data to common period between all stations/networks
- CRN data extends consistently 2011-2019, but HMT stations vary in length. 

In [ ]:
print('Station      Start Date               End Date   ')
print('-----------------------------------------------------')
print('CZC      ', dailyAvg_czc.index[0],'   ', dailyAvg_czc.index[-1])
print('HBG      ', dailyAvg_hbg.index[0],'   ', dailyAvg_hbg.index[-1])   
print('LSN      ', dailyAvg_lsn.index[0],'   ', dailyAvg_lsn.index[-1])
print('PTV      ', dailyAvg_ptv.index[0],'   ', dailyAvg_ptv.index[-1])
print('ROD      ', dailyAvg_rod.index[0],'   ', dailyAvg_rod.index[-1])
print('WLS      ', dailyAvg_wls.index[0],'   ', dailyAvg_wls.index[-1])
print('BDG      ', sm_Bodega["Date"].iloc[0],'   ', sm_Bodega["Date"].iloc[-1])
print('MRC      ', sm_Merced["Date"].iloc[0],'   ', sm_Merced["Date"].iloc[-1])
print('RED      ', sm_Redding["Date"].iloc[0],'   ', sm_Redding["Date"].iloc[-1])
print('STB      ', sm_SantaBarb["Date"].iloc[0],'   ', sm_SantaBarb["Date"].iloc[-1])
print('YSM      ', sm_Yosemite["Date"].iloc[0],'   ', sm_Yosemite["Date"].iloc[-1])


In [ ]:
startDate = '2012-05-23'
endDate   = '2017-12-31'

# Select time period in NOAA HMT data 
CZC_sel = dailyAvg_czc.loc[(dailyAvg_czc.index >= startDate)]
HBG_sel = dailyAvg_hbg.loc[(dailyAvg_hbg.index >= startDate)]
LSN_sel = dailyAvg_lsn.loc[(dailyAvg_lsn.index >= startDate)]
PTV_sel = dailyAvg_ptv.loc[(dailyAvg_ptv.index >= startDate)]
ROD_sel = dailyAvg_rod.loc[(dailyAvg_rod.index >= startDate)]
WLS_sel = dailyAvg_wls.loc[(dailyAvg_wls.index >= startDate)]

# Select time period in soil moisture CRN data 
BDG_sel = sm_Bodega.loc[((sm_Bodega.Date       >= startDate) & (sm_Bodega.Date <= endDate))]
MRC_sel = sm_Merced.loc[((sm_Merced.Date       >= startDate) & (sm_Merced.Date <= endDate))] 
RED_sel = sm_Redding.loc[((sm_Redding.Date     >= startDate) & (sm_Redding.Date <= endDate))]
STB_sel = sm_SantaBarb.loc[((sm_SantaBarb.Date >= startDate) & (sm_SantaBarb.Date <= endDate))]
YSM_sel = sm_Yosemite.loc[((sm_Yosemite.Date   >= startDate) & (sm_Yosemite.Date <= endDate))]

# Select the same time period for everything in soil temperature arrays as well 
BDG_AllSel = BDG_allCRN.loc[((BDG_allCRN.Date   >= startDate) & (BDG_allCRN.Date <= endDate))]
MRC_AllSel = MRC_allCRN.loc[((MRC_allCRN.Date   >= startDate) & (MRC_allCRN.Date <= endDate))] 
RED_AllSel = RED_allCRN.loc[((RED_allCRN.Date   >= startDate) & (RED_allCRN.Date <= endDate))]
STB_AllSel = STB_allCRN.loc[((STB_allCRN.Date   >= startDate) & (STB_allCRN.Date <= endDate))]
YSM_AllSel = YSM_allCRN.loc[((YSM_allCRN.Date   >= startDate) & (YSM_allCRN.Date <= endDate))]

#Handle missing data 
BDG_AllSel.loc[(BDG_AllSel.ST_10_Daily<=-9999.0), :] = np.nan
MRC_AllSel.loc[(MRC_AllSel.ST_10_Daily<=-9999.0), :] = np.nan
RED_AllSel.loc[(RED_AllSel.ST_10_Daily<=-9999.0), :] = np.nan
STB_AllSel.loc[(STB_AllSel.ST_10_Daily<=-9999.0), :] = np.nan
YSM_AllSel.loc[(YSM_AllSel.ST_10_Daily<=-9999.0), :] = np.nan

#Define date array
dates      = BDG_sel.Date


## 2. Center the data
Remove the mean from each station individually in terms of soil moisture and temperature (at 10cm here). 

In [ ]:
# Center each timeseries of soil MOISTURE (i.e., remove the mean)

CZC_cent = CZC_sel.soilM_10 - np.nanmean(CZC_sel.soilM_10)
HBG_cent = HBG_sel.soilM_10 - np.nanmean(HBG_sel.soilM_10)
LSN_cent = LSN_sel.soilM_10 - np.nanmean(LSN_sel.soilM_10)
PTV_cent = PTV_sel.soilM_10 - np.nanmean(PTV_sel.soilM_10)
ROD_cent = ROD_sel.soilM_10 - np.nanmean(ROD_sel.soilM_10)
WLS_cent = WLS_sel.soilM_10 - np.nanmean(WLS_sel.soilM_10)

BDG_cent = BDG_sel.sm_10cm - np.nanmean(BDG_sel.sm_10cm)
MRC_cent = MRC_sel.sm_10cm - np.nanmean(MRC_sel.sm_10cm)
RED_cent = RED_sel.sm_10cm - np.nanmean(RED_sel.sm_10cm)
STB_cent = STB_sel.sm_10cm - np.nanmean(STB_sel.sm_10cm)
YSM_cent = YSM_sel.sm_10cm - np.nanmean(YSM_sel.sm_10cm)

In [ ]:
# Center each timeseries of soil TEMPERATURE (i.e., remove the mean)

CZC_centST = CZC_sel.soilT_10 - np.nanmean(CZC_sel.soilT_10)
HBG_centST = HBG_sel.soilT_10 - np.nanmean(HBG_sel.soilT_10)
LSN_centST = LSN_sel.soilT_10 - np.nanmean(LSN_sel.soilT_10)
PTV_centST = PTV_sel.soilT_10 - np.nanmean(PTV_sel.soilT_10)
ROD_centST = ROD_sel.soilT_10 - np.nanmean(ROD_sel.soilT_10)
WLS_centST = WLS_sel.soilT_10 - np.nanmean(WLS_sel.soilT_10)

BDG_centST = BDG_AllSel.ST_10_Daily - np.nanmean(BDG_AllSel.ST_10_Daily)
MRC_centST = MRC_AllSel.ST_10_Daily - np.nanmean(MRC_AllSel.ST_10_Daily)
RED_centST = RED_AllSel.ST_10_Daily - np.nanmean(RED_AllSel.ST_10_Daily)
STB_centST = STB_AllSel.ST_10_Daily - np.nanmean(STB_AllSel.ST_10_Daily)
YSM_centST = YSM_AllSel.ST_10_Daily - np.nanmean(YSM_AllSel.ST_10_Daily)

In [1]:
#Create large array for soil moisture
smArray = np.asarray([CZC_cent, HBG_cent, LSN_cent, PTV_cent, ROD_cent, WLS_cent,
                      BDG_cent, MRC_cent, RED_cent, STB_cent, YSM_cent])

NameError: name 'np' is not defined

In [ ]:
#Create large array for soil temperature
stArray = np.asarray([CZC_centST, HBG_centST, LSN_centST, PTV_centST, ROD_centST, WLS_centST,
                      BDG_centST, MRC_centST, RED_centST, STB_centST, YSM_centST])

## 3. Compute average annual cycle 
- Leap year in 2016 is ignored here (assuming a constant 365-day year) 
- Soil data is first split into years, then each calendar day is averaged over the 5-6 year period available. 

In [ ]:
#Find the index of the first day of each year 
yrFind   = np.arange(2013,2018)
yrStarts = np.empty([5])

for iStart in range(len(yrFind)):
    yrStarts[iStart] = np.where(CZC_sel.index == str(yrFind[iStart])+'-01-01')[0]

#Ignore leap year day Feb 29 in 2016
dateIgnore = np.where(CZC_sel.index == '2016-02-29')

In [ ]:
#Re-define each station's record to be [yr x day] rather than [days]
# May 23rd is the 144th day in a year 

#Define initially empty arrays to store data in 
yearlySM = np.full([11,6,365],np.nan) #Dimensions of [station, yr, day of year]
yearlyST = np.full([11,6,365],np.nan) #Dimensions of [station, yr, day of year]

for iYr in range(6):
    #Handle the first year (doesn't start in Jan)
    if iYr==0:
        yearlySM[:,0,142::] = smArray[:, 0:int(yrStarts[0])]
        yearlyST[:,0,142::] = stArray[:, 0:int(yrStarts[0])]
    #Handle leap year - it's annoying
    elif yrFind[iYr-1]==2016: 
        #Feb 29th is the 60th day of the year, so index=59
        yearlySM[:,iYr,0:59] = smArray[:, int(yrStarts[iYr-1]):int(dateIgnore[0])]
        yearlySM[:,iYr,59::] = smArray[:, int(dateIgnore[0])+1:int(yrStarts[iYr])]
        yearlyST[:,iYr,0:59] = stArray[:, int(yrStarts[iYr-1]):int(dateIgnore[0])]
        yearlyST[:,iYr,59::] = stArray[:, int(dateIgnore[0])+1:int(yrStarts[iYr])]
    #Handle the last year case 
    elif iYr==5:   #If last year of data
        yearlySM[:,iYr,:]    = smArray[:,int(yrStarts[iYr-1])::]
        yearlyST[:,iYr,:]    = stArray[:,int(yrStarts[iYr-1])::]
    else: 
        yearlySM[:,iYr,:]    = smArray[:,int(yrStarts[iYr-1]):int(yrStarts[iYr])]
        yearlyST[:,iYr,:]    = stArray[:,int(yrStarts[iYr-1]):int(yrStarts[iYr])]
        

Get the *average* annual cycle by averaging over the year axis

In [ ]:
#Get annual average cycle 
annCycle_SM = np.nanmean(yearlySM,axis=1)
annCycle_ST = np.nanmean(yearlyST,axis=1)


## 4. Remove the annual cycle

In [ ]:
# Remove annual cycle from soil MOISTURE data 

annCycle_SM_allDays = np.full([11,len(CZC_sel.index)],np.nan)

for iYr in range(len(yrStarts)+1): 
    #Handle case of first year(starts in May)
    if iYr==0:
        annCycle_SM_allDays[:,0:int(yrStarts[0])]   = smArray[:,0:int(yrStarts[0])]-annCycle_SM[:,142::]
    
    #Handle case of the last year 
    elif iYr==len(yrStarts):
        annCycle_SM_allDays[:,int(yrStarts[iYr-1])::] = smArray[:,int(yrStarts[iYr-1])::]-annCycle_SM[:,:]
    
    #Handle leap year case
    elif iYr==4:
        annCycle_SM_allDays[:,int(yrStarts[iYr-1]):int(yrStarts[iYr-1]+59)] = smArray[:,int(yrStarts[iYr-1]):int(dateIgnore[0])]-annCycle_SM[:,0:59]
        annCycle_SM_allDays[:,int(yrStarts[iYr-1]+60):int((yrStarts[iYr]))] = smArray[:,int(dateIgnore[0]+1):int(yrStarts[iYr])]-annCycle_SM[:,59::]
    
    #Handle other years 
    else: 
        annCycle_SM_allDays[:,int(yrStarts[iYr-1]):int((yrStarts[iYr]))]    = smArray[:,int(yrStarts[iYr-1]):int(yrStarts[iYr])]-annCycle_SM[:,:]



In [ ]:
# Remove annual cycle from soil TEMPERATURE data 

annCycle_ST_allDays = np.full([11,len(CZC_sel.index)],np.nan)

for iYr in range(len(yrStarts)+1): 
    #Handle case of first year(starts in May)
    if iYr==0:
        annCycle_ST_allDays[:,0:int(yrStarts[0])]   = stArray[:,0:int(yrStarts[0])]-annCycle_ST[:,142::]
        
    #Handle case of the last year 
    elif iYr==len(yrStarts):
        annCycle_ST_allDays[:,int(yrStarts[iYr-1])::] = stArray[:,int(yrStarts[iYr-1])::]-annCycle_ST[:,:]
    
    #Handle leap year case
    elif iYr==4:
        annCycle_ST_allDays[:,int(yrStarts[iYr-1]):int(yrStarts[iYr-1]+59)] = stArray[:,int(yrStarts[iYr-1]):int(dateIgnore[0])]-annCycle_ST[:,0:59]
        annCycle_ST_allDays[:,int(yrStarts[iYr-1]+60):int((yrStarts[iYr]))] = stArray[:,int(dateIgnore[0]+1):int(yrStarts[iYr])]-annCycle_ST[:,59::]
    
    #Handle other years 
    else: 
        annCycle_ST_allDays[:,int(yrStarts[iYr-1]):int((yrStarts[iYr]))]    = stArray[:,int(yrStarts[iYr-1]):int(yrStarts[iYr])]-annCycle_ST[:,:]
        


## 5. Write out data (centered with annual cycle removed) to text file 

In [ ]:
# # -- Write file with soil MOISTURE info 

#Write out the Eigenvalues and vectors to a file too 
fileName = '/home/mfowler/Python/textFiles/CombinedNetworks/SM_withoutAvgAnnCycle.txt'

#Replacing missing values with -999.0
annCycle_SM_allDays[~np.isfinite(annCycle_SM_allDays)] = -999.0

with open(fileName,'w+') as f: 
    for iT in range(len(dates)): 
        f.write("%s %f %f %f %f %f %f %f %f %f %f %f \n" % (np.asarray(dates)[iT],     annCycle_SM_allDays[0,iT],
                                                            annCycle_SM_allDays[1,iT], annCycle_SM_allDays[2,iT],
                                                            annCycle_SM_allDays[3,iT], annCycle_SM_allDays[4,iT], 
                                                            annCycle_SM_allDays[5,iT], annCycle_SM_allDays[6,iT],
                                                            annCycle_SM_allDays[7,iT], annCycle_SM_allDays[8,iT],
                                                            annCycle_SM_allDays[9,iT], annCycle_SM_allDays[10,iT]) )



In [ ]:
# # -- Write file with soil TEMPERATURE information 

#Write out the Eigenvalues and vectors to a file too 
fileName = '/home/mfowler/Python/textFiles/CombinedNetworks/ST_withoutAvgAnnCycle.txt'

#Replacing missing values with -999.0
annCycle_ST_allDays[~np.isfinite(annCycle_ST_allDays)] = -999.0

with open(fileName,'w+') as f: 
    for iT in range(len(dates)): 
        f.write("%s %f %f %f %f %f %f %f %f %f %f %f \n" % (np.asarray(dates)[iT],     annCycle_ST_allDays[0,iT],
                                                            annCycle_ST_allDays[1,iT], annCycle_ST_allDays[2,iT],
                                                            annCycle_ST_allDays[3,iT], annCycle_ST_allDays[4,iT], 
                                                            annCycle_ST_allDays[5,iT], annCycle_ST_allDays[6,iT],
                                                            annCycle_ST_allDays[7,iT], annCycle_ST_allDays[8,iT],
                                                            annCycle_ST_allDays[9,iT], annCycle_ST_allDays[10,iT]) )



## 6. Get z-scores
- This section on is structured to first read in the above text files, then keep on computing based on that. (Typically quicker than needing to run the whole thing again). 

In [3]:
#Read in the data that has the average annual cycle removed, skipping the date column
fileName_SM    = '/home/mfowler/Python/textFiles/CombinedNetworks/SM_withoutAvgAnnCycle.txt'
SMdata         = pd.read_csv(fileName_SM, sep="\s+", header=None,usecols=np.arange(11)+1)
SMdata.columns = stationIDs

fileName_ST    = '/home/mfowler/Python/textFiles/CombinedNetworks/ST_withoutAvgAnnCycle.txt'
STdata         = pd.read_csv(fileName_ST, sep="\s+", header=None,usecols=np.arange(11)+1) 
STdata.columns = stationIDs

In [4]:
#Combine the two variables together (time x station/var)
comboArr          = np.full([np.shape(np.asarray(SMdata))[0],np.shape(np.asarray(SMdata))[1]*2],np.nan)
comboArr[:,0:11]  = STdata
comboArr[:,11:22] = SMdata

#Put dimensions in right order (station/var x time)
allArray = np.transpose(np.asarray(comboArr))

# Also get the transpose of the combined matrix 
allTranspose = np.transpose(allArray)

print('Shape of data:      ', np.shape(allArray))
print('Shape of transpose: ', np.shape(allTranspose))


Shape of data:       (22, 2049)
Shape of transpose:  (2049, 22)


In [5]:
# -- Get the z-scores for each station/variable -- #
nTimes = len(SMdata.CZC)

#Properly set missing values to NaN from -999 
allArray[allArray<=-999]         = np.nan
allTranspose[allTranspose<=-999] = np.nan

#Define empty arrays
normAll   = np.full([len(stationIDs)*2, nTimes], np.nan)
normAll_T = np.full([nTimes, len(stationIDs)*2], np.nan)

#Remove the mean from each station record
for i in range(len(stationIDs)*2):
    normAll[i,:]  = (allArray[i,:]     - np.nanmean(allArray[i,:]))/np.nanstd(allArray[i,:])
    normAll_T[:,i]= (allTranspose[:,i] - np.nanmean(allTranspose[:,i]))/np.nanstd(allTranspose[:,i])



/home/mfowler/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less_equal
  


## 7. Linear Regression
- Based on contemporaneous covariance matrix 

In [6]:
#Get contemporaneous covariance matrix
nDat = len(normAll)

c0   = np.full([nDat,nDat],np.nan)
for iR in range(nDat):
    for iC in range(nDat): 
        c0[iR,iC] = np.nansum(normAll[iR,:]*normAll_T[:,iC])/np.nansum(np.isfinite(normAll[iR,:]*normAll_T[:,iC]))
        

In [7]:
# Now we can get the regression coefficients from that matrix, c0 

MM = c0[11:22,11:22]  #Bottom right quadrant
TM = c0[0:11,11:22]   #Top right quadrant 

MM_inv = LA.inv(MM)   #Get the inverse of the moisture covariance matrix  

regr   = np.dot(TM,MM_inv)  #Regression coefficients calcualted as product of these two matrices 


In [8]:
# Sum manually in order to do matrix multiplication and ignore missing values 

# Isolate ST and SM z-scores 
ST = normAll[0:11,:]
SM = normAll[11:22,:]

# Create empty arrays to hold T and M as estimated via regression 
ST_hat = np.zeros([len(regr),len(ST[0,:])])
SM_hat = np.zeros([len(regr),len(SM[0,:])])

# iterate through rows of X
for i in range(len(regr)):
   # iterate through columns of Y
   for j in range(len(ST[0,:])):
       # iterate through rows of Y
       for k in range(len(ST)):
            ST_hat[i,j] += np.nansum(regr[i,k] * SM[k,j])
            SM_hat[i,j] += np.nansum(regr[i,k] * ST[k,j])


In [10]:
# Compute the residual of temperature
#   This removes the part of temperature that is already explained by the moisture field 
resid = ST - ST_hat


In [11]:
# -- Remove the mean again (center the data) -- #
resid_T = np.transpose(resid)
print(np.shape(resid))

#Define empty arrays
normResid   = np.full([len(stationIDs), nTimes], np.nan)
normResid_T = np.full([nTimes, len(stationIDs)], np.nan)

#Remove the mean from each station record
for i in range(len(stationIDs)):
    normResid[i,:]   = resid[i,:]   - np.nanmean(resid[i,:])
    normResid_T[:,i] = resid_T[:,i] - np.nanmean(resid_T[:,i])
    

(11, 2049)


In [12]:
# Replace the normalized S.T. with the normalized residual of S.T. in the full combined array 
normAll[0:11,:] = normResid
normAll_T       = np.transpose(normAll)


## 8. Save new array of daily, anomalous, residual soil temperature and soil moisture 

In [17]:
# Save the array with residaul S.T. and S.M. z-scores in it 
pickle.dump( normAll, open( "/home/mfowler/Python/pickleFiles/zScores_residST_SM.p", "wb" ) )
